Ниже приведен пример PHP функции для генерации координат, определение региона. Данная функция так же используется для генерации признаков при создании заказа.

In [ ]:
## PHP код
public function defineDeliveryInfo ($order_id) {

        $model = new waModel();

        $query = 'SELECT * FROM shop_order_params WHERE order_id = ' . $order_id;
        $result = $model->query($query)->fetchAll();

        $geo = '';
        $geo_lat = '';
        $geo_lon = '';
        $is_insert = 1;
        $geo_result = '';
        
        foreach ($result as $items) {
            if ($items['name'] == 'shipping_address.street' && !empty($items['value'])) {
                $param['params']['shipping_address.street'] = $items['value'];
            }
            if ($items['name'] == 'shipping_address.city' && !empty($items['value'])) {
                $param['params']['shipping_address.city'] = $items['value'];
            }
            if ($items['name'] == 'shipping_address.lat') {
                $geo = $items['value'];
                $is_insert = 0;
            }
            if ($items['name'] == 'shipping_address.lat' && !empty($items['value'])) {
                $geo_lat = $items['value'];
            }
            if ($items['name'] == 'shipping_address.lng' && !empty($items['value'])) {
                $geo_lon = $items['value'];
            }
            if ($items['name'] == 'geo_result') {
                $geo_result = $items['value'];
            }
        }

        if (!empty($param['params']['shipping_address.street'])) {
            $address = $param['params']['shipping_address.street'];
        } else {
            $address = $param['params']['shipping_address.city'];
        }

        if (empty($geo) && empty($geo_result)) {
            $apiKey = '###';
            $headers = array('Content-Type: application/json', 'Accept: application/json', 'Authorization: Token ' . $apiKey, 'X-Secret: ##');
            $url = 'https://cleaner.dadata.ru/api/v1/clean/address';

            $result = self::curlRequest($url, 'POST', array($address), $headers);

            if (isset($result[0])) {

                $data = $result[0];

                $geo_lat = $data['geo_lat'];
                $geo_lon = $data['geo_lon'];
                $province = $data['region_with_type'];

                $region_id = str_replace('00000000000', '', $data['region_kladr_id']);

                if ($is_insert) {

                    $query = 'UPDATE shop_order_params SET value = \'' . $region_id . '\' WHERE order_id = ' . $order_id . ' AND name = \'shipping_address.region\'';
                    $model->exec($query);

                    if (!empty($geo_lat) && !empty($geo_lon)) {
                        $query = 'INSERT INTO shop_order_params (`order_id`, `name`, `value`) VALUES (' . $order_id . ', \'shipping_address.oblast\', \'' . $province . '\')';
                        $model->exec($query);

                        $query = 'INSERT INTO shop_order_params (`order_id`, `name`, `value`) VALUES (' . $order_id . ', \'shipping_address.lat\', ' . $geo_lat . ')';
                        $model->exec($query);

                        $query = 'INSERT INTO shop_order_params (`order_id`, `name`, `value`) VALUES (' . $order_id . ', \'shipping_address.lng\', ' . $geo_lon . ')';
                        $model->exec($query);
                    } else {
                        $query = 'INSERT INTO shop_order_params (`order_id`, `name`, `value`) VALUES (' . $order_id . ', \'geo_result\', \'Не удалось определить координаты\')';
                        $model->exec($query);
                    }

                } else {
                    $query = 'UPDATE shop_order_params SET value = \'' . $geo_lat . '\' WHERE order_id = ' . $order_id . ' AND name = \'shipping_address.lat\' ';

                    $model->exec($query);
                    $query = 'UPDATE shop_order_params SET value = \'' . $geo_lon . '\' WHERE order_id = ' . $order_id . ' AND name = \'shipping_address.lng\' ';
                    $model->exec($query);

                }
            }
        }

        return array('lat' => $geo_lat, 'lon' => $geo_lon);
    }

In [ ]:
# PHP curl функция для запроса
public function curlRequest ($url, $method='GET', $post=array(), $headers=array("Content-type: application/json")) {
        if (function_exists('curl_init')) {

            $ch = curl_init($url);

            curl_setopt($ch, CURLOPT_RETURNTRANSFER, true);
            curl_setopt($ch, CURLOPT_SSL_VERIFYPEER, false);

            if ($method == 'POST') {
                curl_setopt($ch, CURLOPT_POST, true);
                curl_setopt($ch, CURLOPT_HTTPHEADER, $headers);
                curl_setopt($ch, CURLOPT_POSTFIELDS, json_encode($post));
            }

            curl_setopt($ch, CURLOPT_CONNECTTIMEOUT, 5);

            $resp = curl_getinfo($ch);
            $result = curl_exec($ch);


            $result = json_decode($result, true);

            curl_close($ch);
        }
        return $result;
    }
}    

Функция для получения дистанции и времени в пути от адреса склада до адреса клиента. Используется после получения гео кординат.

In [ ]:
public function routes (coordinate) {

    $query = 'SELECT * FROM shop_order_params WHERE order_id = ' . $order_id;
    $model = new waModel();

    $order_params = $model->query($query)->fetchAll();

    $geo_stock_lat = '55.8328';
    $geo_stock_lng = '38.2725';
    $geo_client_lat = '';
    $geo_client_lng = '';
    $distance = 0;
    $duration = 0;
    $region_id = '';

    foreach ($order_params as $item) {
        if ($item['name'] == 'shipping_address.lat') {
            $geo_client_lat = $item['value'];
        }
        if ($item['name'] == 'shipping_address.lng') {
            $geo_client_lng = $item['value'];
        }
        if ($item['name'] == 'distance') {
            $distance = $item['value'];
        }
        if ($item['name'] == 'duration') {
            $duration = $item['value'];
        }
        if ($item['name'] == 'shipping_address.region') {
            $region_id = $item['value'];
        }
    }

    if ($geo_stock_lat && $geo_stock_lng && $geo_client_lat && $geo_client_lng && empty($distance)) {
        $url = 'http://router.project-osrm.org/route/v1/driving/' . $geo_stock_lng . ',' . $geo_stock_lat . ';' . $geo_client_lng . ',' . $geo_client_lat . '?overview=false';
        $result = self::curlRequest($url);
        if (isset($result['routes'][0]['duration'])) {
            $query = 'INSERT INTO shop_order_params (`order_id`, `name`, `value`) VALUES (' . $order_id . ', \'duration\', ' . $result['routes'][0]['duration'] . ')';
            $model->exec($query);

            $query = 'INSERT INTO shop_order_params (`order_id`, `name`, `value`) VALUES (' . $order_id . ', \'distance\', ' . $result['routes'][0]['distance'] . ')';
            $model->exec($query);

            $duration = $result['routes'][0]['duration'];
            $distance = $result['routes'][0]['distance'];
        }
    }
    return array($duration, $distance)
}